In [1]:
import bw2data as bd
import bw2io as bi
import pandas as pd

In [2]:
bd.projects.set_current("bw25_plca_grid_expansion")

## Creating background databases

### Getting ecoinvent

In [3]:
# bi.import_ecoinvent_release(
#     version="3.10",
#     system_model="cutoff",
#     username="XXXXXXXXXXXXXX",
#     password="XXXXXXXXXXXXXX",
# )

In [4]:
# bd.databases

### Creating prospective databases

In [5]:
# from premise import *

In [6]:
# ndb = NewDatabase(
#     scenarios=[
#         {"model":"image", "pathway":"SSP2-RCP26", "year":2023},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2025},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2030},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2035},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2037},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2040},
#     ],
#     source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
#     source_version="3.10", # <-- version of ecoinvent. Can be "3.8", "3.9" or "3.10". Must be a string.
#     key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
#     biosphere_name='ecoinvent-3.10-biosphere', # <-- name of the biosphere database in the BW2 project. Must be a string.
# )

In [7]:
# ndb.update()

In [8]:
# db_names = [
#     'ei310_SSP2_RCP26_2020',
#     'ei310_SSP2_RCP26_2025',
#     'ei310_SSP2_RCP26_2030',
#     'ei310_SSP2_RCP26_2035',
#     'ei310_SSP2_RCP26_2037',
#     'ei310_SSP2_RCP26_2040',
# ]

# ndb.write_db_to_brightway(db_names)


In [9]:
# bd.databases

## Creating the grid database

### Preparation

In [10]:
def load_inputs_from_background(database_name: str):
    db = bd.Database(database_name)
    return {
        "glass_wool_mat": db.get(name="market for glass wool mat", location="GLO"),
        "porcelain": db.get(name="market for ceramic tile", location="GLO"),
        "epoxy": db.get(name="market for epoxy resin, liquid", location="RER"),
        "brass": db.get(name="market for brass", location="RoW"),
        "paint": db.get(name="market for electrostatic paint", location="GLO"),
        "paper": db.get(name="market for paper, melamine impregnated", location="RER"),
        "rubber": db.get(name="market for synthetic rubber", location="GLO"),
        "sulfur_hexafluoride": db.get(
            name="market for sulfur hexafluoride, liquid", location="RER"
        ),
        "lead": db.get(name="market for lead", location="GLO"),
        "bronze": db.get(name="market for bronze", location="GLO"),
        "polypropylene": db.get(
            name="market for polypropylene, granulate", location="GLO"
        ),
        "steel_sheet": db.get(name="market for sheet rolling, steel", location="GLO"),
        "steel_hot_rolled": db.get(
            name="market for steel, low-alloyed, hot rolled", location="GLO"
        ),
        "steel_lowalloyed": db.get(
            name="market for steel, low-alloyed", location="GLO"
        ),
        "steel_unalloyed": db.get(name="market for steel, unalloyed", location="GLO"),
        "transformer_oil": db.get(name="market for lubricating oil", location="RER"),
        "aluminium_wrought_alloy": db.get(
            name="market for aluminium, wrought alloy", location="GLO"
        ),
        "aluminium_sheet": db.get(
            name="market for sheet rolling, aluminium", location="GLO"
        ),
        "aluminium_cast": db.get(
            name="market for aluminium, cast alloy", location="GLO"
        ),
        "copper": db.get(name="market for copper, cathode", location="GLO"),
        "copper_wire": db.get(name="wire drawing, copper", location="RER"),
        "glass_fibre": db.get(name="market for glass fibre", location="GLO"),
        "kraft_paper": db.get(name="market for kraft paper", location="RER"),
        "softwood": db.get(
            name="market for sawnwood, softwood, raw, dried (u=20%)", location="RER"
        ),
        "concrete": db.get(
            name="market for concrete, normal strength", location="RoW"
        ),  # density of 2200 kg/m3 assumed for conversion, see Itten et al.
        "zinc": db.get(name="market for zinc", location="GLO"),
        "waste_concrete": db.get(
            name="market for waste concrete", location="Europe without Switzerland"
        ),
        "waste_polyethylene": db.get(
            name="market for waste polyethylene", location="DE"
        ),
        "waste_oil": db.get(
            name="market for waste mineral oil", location="Europe without Switzerland"
        ),
        "excavation": db.get(
            name="market for excavation, hydraulic digger", location="GLO"
        ),
        "polyester_resin": db.get(
            name="market for polyester resin, unsaturated", location="RER"
        ),
        "steel_chromium_steel": db.get(
            name="market for steel, chromium steel 18/8", location="GLO"
        ),
        "polycarbonate": db.get(name="market for polycarbonate", location="RER"),
        "wood": db.get(name="market for fibreboard, hard", location="RER"),
        "cement_unspecified": db.get(
            name="market for cement, unspecified", location="Europe without Switzerland"
        ),
        "glass_tube_borosilicate": db.get(
            name="market for glass tube, borosilicate", location="GLO"
        ),
        "extrusion_plastic_pipes": db.get(
            name="market for extrusion, plastic pipes", location="GLO"
        ),
        "polyethylene": db.get(
            name="market for polyethylene, low density, granulate", location="GLO"
        ),
        "copper_wire_drawing": db.get(
            name="market for wire drawing, copper", location="GLO"
        ),
    }

In [11]:
# (code, name, unit) of grid components
COMPONENT_INFO = [
    ("disconnector", "Disconnector", "unit"),
    (
        "land_cable_oil_cu_150kv",
        "Land cable, oil insulated, copper, 150kV",
        "kilometer",
    ),
    #  ('live_tank_circuit_breaker', 'Live tank circuit breaker', 'unit'),
    ("overhead_line_400kv", "Overhead line, 400kV", "kilometer"),
    ("overhead_line_150kv", "Overhead line, 150kV", "kilometer"),
    ("overhead_line_10kv", "Overhead line, 10kV", "kilometer"),
    ("overhead_line_04kv", "Overhead line, 0.4kV", "kilometer"),
    ("overhead_line_HVDC", "Overhead line HVDC", "kilometer"),
    ("land_cable_oil_cu_HVDC", "Land cable, oil insulated, copper, HVDC", "kilometer"),
    #  ('plug_and_switch_system', 'Plug and switch system', 'unit'),
    #  ('surge_arrester', 'Surge arrester', 'unit'),
    ("transformer_250mva", "Transformer, 250MVA", "unit"),
    ("transformer_40mva", "Transformer, 40MVA", "unit"),
    ("transformer_315kva", "Transformer, 315kVA", "unit"),
    (
        "land_cable_vpe_al_04kv",
        "Land cable, vpe insulated, aluminium, 0.4kV",
        "kilometer",
    ),
    (
        "land_cable_vpe_al_10kv",
        "Land cable, vpe insulated, aluminium, 10kV",
        "kilometer",
    ),
    (
        "land_cable_vpe_al_10kv",
        "Land cable, vpe insulated, aluminium, 10kV",
        "kilometer",
    ),
    ("land_cable_epr_cu_11kv", "Land cable, epr insulated, copper, 11kV", "kilometer"),
    (
        "land_cable_vpe_al_50kv",
        "Land cable, vpe insulated, aluminium, 50kV",
        "kilometer",
    ),
    ("land_cable_vpe_cu_1kv", "Land cable, vpe insulated, copper, 1kV", "kilometer"),
    ("gas_insulated_switchgear_420kv", "Gas insulated switchgear, 420kV", "unit"),
    ("substation_lv", "Substation, LV", "unit"),
    ("substation_mv", "Substation, MV", "unit"),
    # ('office_building', 'Office building for grid operation', 'unit'),
    #  ('uniswitch', 'Uniswitch', 'unit')
]

In [12]:

def create_component_nodes(base_database_name: str, component_database_name: str):
    if component_database_name in bd.databases:
        del bd.databases[component_database_name]
    db_components = bd.Database(component_database_name)
    db_components.register()
    
    INPUT_NODES = load_inputs_from_background(base_database_name)
    
    for code, name, unit in COMPONENT_INFO:
        try:
            component = db_components.new_node(
                code=code, name=name, unit=unit, **{"reference product": name}
            )
        except:
            db_components.get(code).delete()
            component = db_components.new_node(
                code=code, name=name, unit=unit, **{"reference product": name}
            )
            component.save()
        component.save()
        component.new_edge(
            name=component["name"], input=component, amount=1, unit=unit, type="production"
        ).save()

        inputs_df = pd.read_csv(f"data/{code}.csv", sep=";", index_col="input")
        for input_code, amount, unit in zip(inputs_df.index, inputs_df.amount, inputs_df.unit):
            component.new_edge(
                input=INPUT_NODES[input_code], amount=amount, type="technosphere"
            ).save()

        if code == "gas_insulated_switchgear_420kv":
            sf6_node = bd.get_node(database="ecoinvent-3.10-biosphere", name="Sulfur hexafluoride", categories=("air",))
            component.new_edge(
                input=sf6_node,
                amount=28.6, # leakage
                type="biosphere",
            ).save()

### Creating component nodes

In [13]:
bd.databases

Databases dictionary with 4 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	grid_components
	grid_components_2020

In [14]:
create_component_nodes("ecoinvent-3.10-cutoff", "grid_components_2020")

In [15]:
bd.databases

Databases dictionary with 4 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	grid_components
	grid_components_2020

### Creating grid nodes

In [16]:
SHARE_OVERHEAD_LINES_EHV_AC = 1
SHARE_OVERHEAD_LINES_EHV_DC = 0.05 
SHARE_OVERHEAD_LINES_HV = 0.9474
SHARE_OVERHEAD_LINES_MV = 0.1847
SHARE_OVERHEAD_LINES_LV = 0.0652

SHARE_AL_CABLES_EHV = 1
SHARE_AL_CABLES_HV = 0.5
SHARE_AL_CABLES_MV = 0.75
SHARE_AL_CABLES_LV = 0.75

components = {
    2023: {
        # EHV AC
        "overhead_line_400kv": SHARE_OVERHEAD_LINES_EHV_AC * 36400,
        # EHV DC
        "overhead_line_HVDC": SHARE_OVERHEAD_LINES_EHV_DC * 2172,
        "land_cable_oil_cu_HVDC": (1 - SHARE_OVERHEAD_LINES_EHV_DC) * 2172,   
        # HV
        "overhead_line_150kv": SHARE_OVERHEAD_LINES_HV * 95200,
        "land_cable_vpe_al_50kv": (1 - SHARE_OVERHEAD_LINES_HV) * SHARE_AL_CABLES_HV * 95200, 
        "land_cable_oil_cu_150kv": (1 - SHARE_OVERHEAD_LINES_HV) * (1 - SHARE_AL_CABLES_HV) * 95200, 
        "transformer_250mva": 709, # EHV -> HV
        # "substation_mv": 709, # same as mv assumed, therefore included below
        # MV
        "overhead_line_10kv": SHARE_OVERHEAD_LINES_MV * 530200,
        "land_cable_vpe_al_10kv": (1 - SHARE_OVERHEAD_LINES_MV) * SHARE_AL_CABLES_MV * 530200,
        "transformer_40mva": 7278, # HV -> MV
        "gas_insulated_switchgear_420kv": 7278,
        "substation_mv": 7278 + 709, 
        # LV
        "overhead_line_04kv": SHARE_OVERHEAD_LINES_LV * 1570100,
        "land_cable_vpe_al_04kv": (1 - SHARE_OVERHEAD_LINES_LV) * SHARE_AL_CABLES_LV * 1570100,
        "land_cable_vpe_cu_1kv": (1 - SHARE_OVERHEAD_LINES_LV) * (1 - SHARE_AL_CABLES_LV) * 1570100,
        "transformer_315kva": 577790, # MV -> LV
        "substation_lv": 577790, 
    },
    2037: {
        # EHV AC
        "overhead_line_400kv": SHARE_OVERHEAD_LINES_EHV_AC * 11823,
        # EHV DC
        "overhead_line_HVDC": SHARE_OVERHEAD_LINES_EHV_DC * 17492,
        "land_cable_oil_cu_HVDC": (1 - SHARE_OVERHEAD_LINES_EHV_DC) * 17492,
        # HV
        "overhead_line_150kv": SHARE_OVERHEAD_LINES_HV * 28307,
        "land_cable_vpe_al_50kv": (1 - SHARE_OVERHEAD_LINES_HV) * SHARE_AL_CABLES_HV * 28307, 
        "land_cable_oil_cu_150kv": (1 - SHARE_OVERHEAD_LINES_HV) * (1 - SHARE_AL_CABLES_HV) * 28307, 
        "transformer_250mva": 221, # EHV -> HV
        # "substation_mv": 709/1.58, # same as mv assumed, therefore included below
        # MV
        "overhead_line_10kv": SHARE_OVERHEAD_LINES_MV * 117593,
        "land_cable_vpe_al_10kv": (1 - SHARE_OVERHEAD_LINES_MV) * SHARE_AL_CABLES_MV * 117593,
        "land_cable_epr_cu_11kv": (1 - SHARE_OVERHEAD_LINES_MV) * (1 - SHARE_AL_CABLES_MV) * 117593,
        "transformer_40mva": 1890, # HV -> MV
        "gas_insulated_switchgear_420kv": 1890,
        "substation_mv": (1890+709)/1.58, 
        
        # LV
        "overhead_line_04kv": SHARE_OVERHEAD_LINES_LV * 375511,
        "land_cable_vpe_al_04kv": (1 - SHARE_OVERHEAD_LINES_LV) * SHARE_AL_CABLES_LV * 375511,
        "land_cable_vpe_cu_1kv": (1 - SHARE_OVERHEAD_LINES_LV) * (1 - SHARE_AL_CABLES_LV) * 375511,
        "transformer_315kva": 133273, # MV -> LV
        "substation_lv": 133273, 
    },    
    2045: {
        # EHV AC
        "overhead_line_400kv": SHARE_OVERHEAD_LINES_EHV_AC * 250,
        # EHV DC
        "overhead_line_HVDC": SHARE_OVERHEAD_LINES_EHV_DC * 4683,
        "land_cable_oil_cu_HVDC": (1 - SHARE_OVERHEAD_LINES_EHV_DC) * 4683,
        # HV
        "overhead_line_150kv": SHARE_OVERHEAD_LINES_HV * 15096,
        "land_cable_vpe_al_50kv": (1 - SHARE_OVERHEAD_LINES_HV) * SHARE_AL_CABLES_HV * 15096, 
        "land_cable_oil_cu_150kv": (1 - SHARE_OVERHEAD_LINES_HV) * (1 - SHARE_AL_CABLES_HV) * 15096, 
        "transformer_250mva": 59, # EHV -> HV
        # "substation_mv": 709, # same as mv assumed, therefore included below
        # MV
        "overhead_line_10kv": SHARE_OVERHEAD_LINES_MV * 64837,
        "land_cable_vpe_al_10kv": (1 - SHARE_OVERHEAD_LINES_MV) * SHARE_AL_CABLES_MV * 64837,
        "transformer_40mva": 1022, # HV -> MV
        "gas_insulated_switchgear_420kv": 1022,
        "substation_mv": (1022+709)/1.58, 
        # LV
        "overhead_line_04kv": SHARE_OVERHEAD_LINES_LV * 198933,
        "land_cable_vpe_al_04kv": (1 - SHARE_OVERHEAD_LINES_LV) * SHARE_AL_CABLES_LV * 198933,
        "land_cable_vpe_cu_1kv": (1 - SHARE_OVERHEAD_LINES_LV) * (1 - SHARE_AL_CABLES_LV) * 198933,
        "transformer_315kva": 71943, # MV -> LV
        "substation_lv": 71943, 
    },
}

Status quo:

In [17]:
db = bd.Database("grid_components_2020")

activity_code = 'grid_status_quo'
activity_name = activity_code

try:
    grid = db.new_activity(code=activity_code, name=activity_name, unit='unit', **{'reference product': activity_name})
except:
    existing_activity = db.get(activity_code)
    existing_activity.delete()
    grid = db.new_activity(code=activity_code, name=activity_name, unit='unit', **{'reference product': activity_name})
grid.save()
grid.new_exchange(input=grid, amount=1, type='production').save()
for key, value in components[2023].items():
    grid.new_exchange(input=db.get(key), amount=value, type='technosphere').save()

### Aggregated material nodes for status quo

In [18]:
material_exchanges = {
    'aluminium': {},
    'steel': {},
    'concrete': {},
    'copper': {},
    'plastics': {}
}

for component_exchange in grid.technosphere():
    for material_exchange in component_exchange.input.technosphere():
        name = material_exchange.input['name'].lower()
        key = None
        if "aluminium" in name:
            key = 'aluminium'
        elif "steel" in name or "iron" in name:
            key = 'steel'
        elif "concrete" in name:
            key = 'concrete'
        elif "copper" in name:
            key = 'copper'
        elif "polyethylene" in name or "polypropylene" in name or "plastic" in name:
            key = 'plastics'

        total_amount = material_exchange.amount * component_exchange.amount
        
        if key:
            if material_exchange.input in material_exchanges[key]:
                material_exchanges[key][material_exchange.input] += total_amount
            else:
                material_exchanges[key][material_exchange.input] = total_amount
        else:
            if material_exchange.input in material_exchanges.setdefault('other', {}):
                material_exchanges['other'][material_exchange.input] += total_amount
            else:
                material_exchanges['other'][material_exchange.input] = total_amount


In [19]:
aggregated_material_nodes = []
for name, subexchanges in material_exchanges.items():
    try:
        mat = db.new_node(
            name=f"aggregated material: {name}"
        )
    except:
        db.get(name).delete()
        mat = db.new_node(
            name=f"aggregated material: {name}"
        )
    mat.save()
    aggregated_material_nodes.append(mat)
    mat.new_exchange(input=mat, amount=1, type="production").save()
    for node, value in subexchanges.items():
        mat.new_exchange(input=node, amount=value, type="technosphere").save()